# Used trained model to predict 

In [114]:
# (the main logic come from /lcrc/group/ATLAS/users/jj/DiHydra_jj/detection_combined/transformer_based_detection/informers/informerrunner.py)

In [115]:
import sys
import site
site.getusersitepackages()
sys.path.append(site.getusersitepackages())
sys.path.append('/lcrc/group/ATLAS/users/jj/DiHydra_jj/transformer_based_detection/informers')
sys.path.append('/lcrc/group/ATLAS/users/jj/DiHydra_jj/detection_combined/utils')


In [138]:
#input for informerrunner.py
#comes from the class of informerrunner self.__init__,  make it to global variables to handle
checkpoint_dir = "/lcrc/group/ATLAS/users/jj/DiHydra_jj/detection_combined/transformer_based_detection/informers/checkpoints"
loss_type='mse'
use_spot_detection=False # for onlince detection only
device= 'cpu'
output_queue = None
setting = '/hlt_dcm_2023_mse_seed_42'
param_file_string= checkpoint_dir + setting + '/model_parameters.json' 
path = checkpoint_dir
model_path = path + setting + '/checkpoint_informer.pth'

#Don't change following parameters
data_x_last = None
anomaly_start = None
anomaly_duration = 0
#nan_output_tolerance_period = {"count": 0, "threshold": 10}
nan_output_tolerance_period = {} # threshold of tolerated consecutive NaN predictions of informer is 10
nan_output_count = 0
predictions_all = []


In [117]:
import json
with open(param_file_string, 'r') as parameter_dict_file:
    parameter_dict = json.load(parameter_dict_file)
parameter_dict['timeenc'] = 0 if parameter_dict['embed'] != 'timeF' else 1

In [118]:
import os
print("Current working directory:", os.getcwd())
os.chdir('/lcrc/group/ATLAS/users/jj/DiHydra_jj')
print("Current working directory:", os.getcwd())

Current working directory: /lcrc/group/ATLAS/users/jj/DiHydra_jj
Current working directory: /lcrc/group/ATLAS/users/jj/DiHydra_jj


In [119]:
import logging
from collections.abc import Callable
import numpy as np
import pandas as pd
import torch
import sys
sys.path.append('/lcrc/group/ATLAS/users/jj/DiHydra_jj/detection_combined')
from transformer_based_detection.informers.models.model import Informer
from transformer_based_detection.informers.utils.datapreprocessor import DataPreprocessor
from detection_combined.utils.exceptions import NonCriticalPredictionException
from utils.spot import SPOT
from anomalyclassification import AnomalyType
#from utils.exceptions import NonCriticalPredictionException
#from utils.anomalyclassification import AnomalyType  # Why cannot find Module by this path?
from detection_combined.utils.anomalyclassification import AnomalyType
from detection_combined.utils.tqdmloggingdecorator import tqdmloggingdecorator

data_preprocessor = DataPreprocessor(parameter_dict,checkpoint_dir)
logger = logging.getLogger(__name__)

In [120]:
print(checkpoint_dir)

/lcrc/group/ATLAS/users/jj/DiHydra_jj/detection_combined/transformer_based_detection/informers/checkpoints


In [121]:
def load_model():
    params = parameter_dict
    model = Informer(params['enc_in'], params['dec_in'], params['c_out'], params['seq_len'],
                     params['label_len'], params['pred_len'], params['factor'], params['d_model'],
                     params['n_heads'], params['e_layers'], params['d_layers'], params['d_ff'],
                     params['dropout'], params['attn'], params['embed'], params['freq'],
                     params['activation'], True, params['distil'], params['mix'], device ).float()
    return model

In [122]:
model = load_model().to(device)

In [123]:
model.eval()
#eval() will switch model to "evaluation mode," 
#but it does not perform the evaluation!
#use model without calling eval()), can lead to: 
#1. Unstable predictions due to dropout layer
#2. Performance issues because of incorrect BatchNorm statistics
#However, in the informer we use layerNorm instead of BatchNorm 

Informer(
  (enc_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(146, 576, kernel_size=(3,), stride=(1,), padding=(1,), padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=6, out_features=576, bias=True)
    )
    (dropout): Dropout(p=0.001, inplace=False)
  )
  (dec_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(146, 576, kernel_size=(3,), stride=(1,), padding=(1,), padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=6, out_features=576, bias=True)
    )
    (dropout): Dropout(p=0.001, inplace=False)
  )
  (encoder): Encoder(
    (attn_layers): ModuleList(
      (0): EncoderLayer(
        (attention): AttentionLayer(
          (inner_attention): ProbAttention(
            (dropout): Drop

In [124]:
def _process_one_batch(dataset_object,
                        batch_x,
                        batch_y,
                        batch_x_mark,
                        batch_y_mark,
                        viz_data):

        batch_x = batch_x.float().to(device)
        batch_y = batch_y.float()

        batch_x_mark = batch_x_mark.float().to(device)
        batch_y_mark = batch_y_mark.float().to(device)

        # Decoder input

        padding = parameter_dict['padding']
        pred_len = parameter_dict['pred_len']
        label_len = parameter_dict['label_len']

        if padding == 0:
            dec_inp = torch.zeros([batch_y.shape[0], pred_len, batch_y.shape[-1]]).float()

        elif padding == 1:
            dec_inp = torch.ones([batch_y.shape[0], pred_len, batch_y.shape[-1]]).float()

        dec_inp = torch.cat([batch_y[:, :label_len, :], dec_inp], dim=1).float().to(device)

        # Encoder - decoder

        outputs, _ = model(batch_x, batch_x_mark, dec_inp, batch_y_mark,
                                                            viz_data=viz_data)

        if parameter_dict['inverse']:
            outputs = dataset_object.inverse_transform(outputs)

        f_dim = 0
        batch_y = batch_y[:, -pred_len:, f_dim:].to(device)

        return outputs, batch_y


In [125]:
@tqdmloggingdecorator
def detect(data: pd.DataFrame, 
           data_preprocessor, 
           parameter_dict, 
           nan_output_tolerance_period, 
           logger, 
           predictions_all, 
           data_x_last, 
           anomaly_start, 
           anomaly_duration, 
           detection_callback):

    data_x, data_y, data_x_mark, data_y_mark = data_preprocessor.process(data)
    timestamp = data.index[-1]

    # Convert data to numpy for visualization
    viz_data = data.to_numpy()[:parameter_dict['seq_len'], :]

    # Process a single batch
    preds, _ = _process_one_batch(data_preprocessor, data_x, data_y, data_x_mark, data_y_mark, viz_data)

    preds = preds.detach().cpu().numpy()

    # Check for NaN predictions
    if np.any(np.isnan(preds)):
        nan_output_count = nan_output_tolerance_period.get("count", 0) + 1
        nan_output_tolerance_period["count"] = nan_output_count

        if nan_output_count >= nan_output_tolerance_period["threshold"]:
            logger.warning('Encountered NaN in Informer predictions')
            logger.error(f'Reached threshold of tolerated consecutive NaN predictions of {nan_output_tolerance_period["threshold"]}')
            raise NonCriticalPredictionException(
                f'Informer reached threshold of tolerated consecutive NaN predictions of {nan_output_tolerance_period["threshold"]}'
            )
        else:
            logger.warning('Encountered NaN in Informer predictions')
            logger.warning(f'Consecutive NaN predictions: {nan_output_count}, tolerated NaN predictions: {nan_output_tolerance_period["threshold"]}')
        return
    else:
        nan_output_tolerance_period["count"] = 0

    l2_dist_detection = False #Only if current time step l2_dist <0.5

    # Calculate L2 distance if data_x_last exists
    if data_x_last is not None:
        l2_dist = np.mean((preds[:, 0, :] - data_y.detach().cpu().numpy()[:, -1, :])**2, 1)[0]
        #l2_dist measure the Euclidean distance between the prediction and the actual real target
        predictions_all.append(l2_dist)

        l2_dist_detection = (l2_dist > 0.5)

        if l2_dist_detection:
            if anomaly_duration == 0:
                anomaly_start = timestamp.strftime('%Y-%m-%d %H:%M:%S')
                logger.warning(f'Transformer-based detection encountered anomaly at timestamp {anomaly_start}')

            detection_callback(0, AnomalyType.TransformerBased, anomaly_start, anomaly_duration)
            anomaly_duration += 1
        else:
            if anomaly_duration != 0:
                anomaly_end = timestamp.strftime('%Y-%m-%d %H:%M:%S')
                logger.warning(f'Transformer-based detection anomaly ended at {anomaly_end}')
            anomaly_duration = 0

    # Update data_x_last for next detection
    data_x_last = data_y.detach().cpu().numpy() #pass current timestep real data（data_y）to data_x_last for next call of detect

    return predictions_all, data_x_last, anomaly_start, anomaly_duration

In [126]:
def get_predictions():
    return np.array(predictions_all)#just convert list to np.array

In [127]:
def register_detection_callback(callback: Callable) -> None:
    detection_callback = callback

In [128]:
#from transformer_based_detection.informers.informerrunner import InformerRunner
#informer_runner = InformerRunner(checkpoint_dir, device='cpu')


# using above defined functions to make prediction


In [129]:
#the following logic come from /lcrc/group/ATLAS/users/jj/DiHydra_jj/detection_combined/offline_detection/informers_localData.py

In [130]:
inp_data_name='/lcrc/group/ATLAS/users/jhoya/DAQ/atlas-hlt-datasets/test_set_dcm_rates_2023.csv'
hlt_data_pd = pd.read_csv(inp_data_name, index_col=0, parse_dates=True)
hlt_data_np = hlt_data_pd.to_numpy()
timestamps = list(hlt_data_pd.index)
tpu_labels = list(hlt_data_pd.columns.values)
output_dir='/lcrc/group/ATLAS/users/jj/DiHydra_jj/detection_combined/transformer_based_detection/informers/JJ_trained_model_predict_output'

In [134]:
from reduceddatabuffer import ReducedDataBuffer 
reduced_data_buffer = ReducedDataBuffer(size=17) #only for MSE loss, size=65 for SMSE

from detection_combined.reduction.medianstdreducer import MedianStdReducer
median_std_reducer = MedianStdReducer('2023')
sys.path.append('/lcrc/group/ATLAS/users/jj/DiHydra_jj/detection_combined/reduction')
from detection_combined.utils.anomalyclassification import AnomalyType
from detection_combined.utils.anomalyregistry import JSONAnomalyRegistry
json_anomaly_registry = JSONAnomalyRegistry(output_dir)
register_detection_callback(json_anomaly_registry.transformer_detection)
detection_callback =json_anomaly_registry.transformer_detection

reduced_data_buffer.set_buffer_filled_callback(
    lambda data: detect(
        data,
        data_preprocessor,
        parameter_dict,
        nan_output_tolerance_period,
        logger,
        predictions_all,
        data_x_last,
        anomaly_start,
        anomaly_duration,
        detection_callback,
    )
)




In [ ]:
from tqdm.contrib.logging import logging_redirect_tqdm
from tqdm.contrib import tzip

with logging_redirect_tqdm():
    for count, (timestamp, data) in enumerate(tzip(timestamps, hlt_data_np)):
        try:
            output_slice =\
                median_std_reducer.reduce_numpy(tpu_labels,
                                                timestamp,
                                                data)
            reduced_data_buffer.push(output_slice)

        except NonCriticalPredictionException:
            break


  0%|          | 0/24018 [00:00<?, ?it/s]